In [1]:
'''
Code source (with some changes):
https://levelup.gitconnected.com/huggingface-transformers-interpretability-with-captum-28e4ff4df234
https://gist.githubusercontent.com/theDestI/fe9ea0d89386cf00a12e60dd346f2109/raw/15c992f43ddecb0f0f857cea9f61cd22d59393ab/explain.py
'''

import torch
import pandas as pd

from torch import tensor 
import transformers
from transformers.pipelines import TextClassificationPipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

import matplotlib.pyplot as plt

import argparse 
import jsonlines
import os 

/tmp/nlp-conda/envs/inspectEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ExplainableTransformerPipeline():
    """Wrapper for Captum framework usage with Huggingface Pipeline"""
    
    def __init__(self, name:str, pipeline: TextClassificationPipeline, device: str):
        self.__name = name
        self.__pipeline = pipeline
        self.__device = device
        self.indToLabel = ["Negative", "Positive"]
        self.labelToInd = {
            "Negative": 0,
            "Positive": 1,
        }
    
    def forward_func(self, inputs: tensor, position = 0):
        """
            Wrapper around prediction method of pipeline
        """
        pred = self.__pipeline.model(inputs,
                       attention_mask=torch.ones_like(inputs))
        return pred[position]
        
    def visualize(self, 
            inputs: list, 
            attributes: list, 
            outfile_path: str,
            pred, 
            label, 
            delta, 
        ):
        """
            Visualization method.
            Takes list of inputs and correspondent attributs for them to visualize in a barplot
        """
        #import pdb; pdb.set_trace()
        attr_sum = attributes.sum(-1) 
        
        attr = attr_sum / torch.norm(attr_sum)

        vis_data_record = visualization.VisualizationDataRecord(
            word_attributions=attr.numpy()[0],
            pred_prob=torch.nn.Softmax()(pred).max().item(),
            pred_class=self.indToLabel[pred.argmax().item()],
            true_class=self.indToLabel[label],
            attr_class=self.indToLabel[pred.argmax().item()],
            attr_score=attributes.sum(),
            raw_input_ids=inputs,
            convergence_score=delta.numpy()[0]
        )

        _ = visualization.visualize_text([vis_data_record])    

    def explain(self, text: str, label: int, outfile_path: str):
        """
            Main entry method. Passes text through series of transformations and through the model. 
            Calls visualization method.
        """
        prediction = self.__pipeline.predict(text)
        inputs = self.generate_inputs(text)
        baseline = self.generate_baseline(sequence_len = inputs.shape[1])
        
        preds = self.forward_func(inputs)

        lig = LayerIntegratedGradients(self.forward_func, getattr(self.__pipeline.model, 'deberta').embeddings)

        attributes, delta = lig.attribute(inputs=inputs,
                                  baselines=baseline,
                                  target = self.__pipeline.model.config.label2id[prediction[0]['label']], 
                                  return_convergence_delta = True)

        # Give a path to save
        self.visualize(
            self.tokenize_inputs(text), 
            attributes.cpu(), 
            outfile_path,
            preds.cpu(),
            label,
            delta
        )

    def tokenize_inputs(self, text: str) -> list:
        return self.__pipeline.tokenizer.tokenize(text)
    
    def generate_inputs(self, text: str) -> tensor:
        """
            Convenience method for generation of input ids as list of torch tensors
        """
        return torch.tensor(self.__pipeline.tokenizer.encode(text, add_special_tokens=False), device = self.__device).unsqueeze(0)
    
    def generate_baseline(self, sequence_len: int) -> tensor:
        """
            Convenience method for generation of baseline vector as list of torch tensors
        """        
        return torch.tensor([self.__pipeline.tokenizer.cls_token_id] + [self.__pipeline.tokenizer.pad_token_id] * (sequence_len - 2) + [self.__pipeline.tokenizer.sep_token_id], device = self.__device).unsqueeze(0)

In [3]:
def main(args):
    tokenizer = AutoTokenizer.from_pretrained(args.model_checkpoint) 
    model = AutoModelForSequenceClassification.from_pretrained(args.model_checkpoint, num_labels=args.num_labels)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    clf = transformers.pipeline("text-classification", 
                                model=model, 
                                tokenizer=tokenizer, 
                                device=device
                                )
    exp_model = ExplainableTransformerPipeline(args.model_checkpoint, clf, device)

    idx=0
    with jsonlines.open(args.a1_analysis_file, 'r') as reader:
        for obj in reader:
            if idx < 7:
                idx += 1
                continue
            print("Explaining {}".format(idx+1))
            exp_model.explain(obj["review"], obj["label"], os.path.join(args.output_dir,f'example_{idx}'))
            idx+=1
    

In [4]:
if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--analsis_dir', default='out', type=str, help='Directory where attribution figures will be saved')
    # parser.add_argument('--model_checkpoint', type=str, default='microsoft/deberta-v3-base', help='model checkpoint')
    # parser.add_argument('--a1_analysis_file', type=str, default='out/a1_analysis_data.jsonl', help='path to a1 analysis file')
    # parser.add_argument('--num_labels', default=2, type=int, help='Task number of labels')
    # parser.add_argument('--output_dir', default='out', type=str, help='Directory where model checkpoints will be saved')    
    # args = parser.parse_args(args=[])
    class Args:
        def __init__(self):
            self.analsis_dir = "out"
            self.model_checkpoint = 'microsoft/deberta-v3-base'
            self.a1_analysis_file = 'out/a1_analysis_data.jsonl'
            self.num_labels = 2
            self.output_dir = 'out'
    args = Args()
    main(args)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/nlp-conda/envs/inspectEnv/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
You shou

Explaining 8


/var/folders/7m/2n3lfqqn4p9cb73mtczw0fy00000gn/T/ipykernel_46134/3653447128.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_prob=torch.nn.Softmax()(pred).max().item(),


Explaining 9


Explaining 10
